# PySPH Tutorial:  Solving a simple problem

*Prabhu Ramachandran*

Department of Aerospace Engineering, IIT Bombay

----------


Let us try to solve a little problem: the motion of an elliptical drop of water.


Initial conditions:

- $\rho = 1.0$
- Particles are inside a circle of $x^2 + y^2 < 1$
- $dx = 0.025, h = 1.3 dx$
- $u = -100 x, v = 100 y $
- Choose $c_s = 1400, dt=5e-6, tf=0.0076$
- Fluid is incompressible.

The velocity field of the initial condition looks as shown in the picture below:

<img width=400 height=400 src="images/elliptical_drop_ic.png"/>

Let us solve this with the WCSPH scheme.



## Solving this with PySPH


1. Subclass the `Application` class (`pysph.solver.application.Application`).
2. Add a `create_particles(self)` method which returns a tuple/list of particle arrays.
3. Add a `create_scheme(self)` method which returns a suitably configured "WCSPH scheme".


### Exercise

- Starting with the example below, create the particles correctly below.  
- Put this in a separate file called `ed.py`. 
- Remember to set the properties: `u, v, rho, h, m`.
- Don't run this from IPython.


In [ ]:
import numpy as np

from pysph.solver.application import Application
from pysph.base.utils import get_particle_array
from pysph.sph.scheme import WCSPHScheme  #  <--- ADDED
from matplotlib import pyplot as plt
from pysph.base.utils import get_particle_array_wcsph



class EllipticalDrop(Application):
    def create_particles(self):
        x = np.linspace(-1,1,100)
        y = np.linspace(-1,1,100)
        xm, ym = np.meshgrid(x, y)
        dx = 0.025
        h = 1.3*dx
        condition = np.where((xm**2 + ym**2) <= 1)
        xm, ym = xm[condition], ym[condition]
        pa = get_particle_array(name='fluid', x=xm, y=ym, rho=1.0, u=-100*xm, v=100*ym, h=h, m = dx*dx)
        self.scheme.setup_properties([pa])
        return [pa]
    
        
    
    def create_scheme(self):
        s = WCSPHScheme(
            ['fluid'], [], dim=2, rho0=1.0, c0=1400,
            h0=1.3*0.025, hdx=1.3, gamma=7.0, alpha=0.1, beta=0.0
        )
        dt = 5e-6
        tf = 0.0076
        s.configure_solver(
            dt=dt, tf=tf,
        )
        return s 
        

    
app = EllipticalDrop(fname='mysol')
app.run()

### Solution

Try solving this yourselves and then compare with the solution below.

DON'T run this yet!

In [ ]:
# %load solutions/ed0.py
import numpy as np
from pysph.base.utils import get_particle_array
from pysph.solver.application import Application


class EllipticalDrop(Application):
    def create_particles(self):
        dx = 0.025
        x, y = np.mgrid[-1.05:1.05:dx, -1.05:1.05:dx]
        mask = x*x + y*y < 1
        x = x[mask]
        y = y[mask]
        rho = 1.0
        h = 1.3*dx
        m = rho*dx*dx
        pa = get_particle_array(
            name='fluid', x=x, y=y, u=-100*x, v=100*y, rho=rho,
            m=m, h=h
        )
        return [pa]


if __name__ == '__main__':
    app = EllipticalDrop(fname='ed')
    app.run()


### Creating the scheme

- Now that the particles are created, let us create the scheme.


In [ ]:
import numpy as np

from pysph.solver.application import Application
from pysph.base.utils import get_particle_array
from pysph.sph.scheme import WCSPHScheme  #  <--- ADDED

class EllipticalDrop(Application):
    def create_particles(self):
        # ...
        self.scheme.setup_properties([pa])  # <--- ADDED
        return pa
        
    def create_scheme(self):
        s = WCSPHScheme(
            ['fluid'], [], dim=2, rho0=1.0, c0=1400,
            h0=1.3*0.025, hdx=1.3, gamma=7.0, alpha=0.1, beta=0.0
        )
        dt = 5e-6
        tf = 0.0076
        s.configure_solver(
            dt=dt, tf=tf,
        )
        return s        


    
if __name__ == '__main__':
    app = EllipticalDrop()
    app.run()

### Exercise

- Modify your example to add the changes made above.
- Run the example as `python ed.py`
- Visualize the output.

### Solution

Try to get the example running, see the solution below if you are stuck.

In [ ]:
# %load solutions/ed.py
import numpy as np
from pysph.base.utils import get_particle_array
from pysph.solver.application import Application
from pysph.sph.scheme import WCSPHScheme


class EllipticalDrop(Application):
    def create_particles(self):
        dx = 0.025
        x, y = np.mgrid[-1.05:1.05:dx, -1.05:1.05:dx]
        mask = x*x + y*y < 1
        x = x[mask]
        y = y[mask]
        rho = 1.0
        h = 1.3*dx
        m = rho*dx*dx
        pa = get_particle_array_wcsph(
            name='fluid', x=x, y=y, u=-100*x, v=100*y, rho=rho,
            m=m, h=h
        )
        self.scheme.setup_properties([pa])
        return [pa]

    def create_scheme(self):
        s = WCSPHScheme(
            ['fluid'], [], dim=2, rho0=1.0, c0=1400,
            h0=1.3*0.025, hdx=1.3, gamma=7.0, alpha=0.1, beta=0.0
        )
        dt = 5e-6
        tf = 0.0076
        s.configure_solver(
            dt=dt, tf=tf,
        )
        return s



app = EllipticalDrop(fname='ed')
app.run()


### Solution discussion

Recall that you can run the example as follows (on a terminal):

    > python ed.py
        
Try this:

    > python ed.py -h
    
You can also run the example to use OpenMP like so:

    > python ed.py --openmp
    
With multiple cores this should produce some speed-up.

The runs will produce output inside the `ed_output` directory.

- You can use `-d output_dir` to have the output generated in the `output_dir` instead.
    
    
You can visualize the output using:

    > pysph view ed_output
    
    
Note that the output directory also contains a very useful log file that is handy when debugging, in this case the file would be `ed_output/ed.log`.  

The output directory contains the data saved at different times in either `*.npz` or `*.hdf5` files depending on your installation.  The next chapter explores post-processing a little.

### Running the example from inside IPython

**NOTE:**  If you want to run the example on an IPython notebook you may do so, but you will need to create the app instance as follows::

```
if __name__ == '__main__':
    app = EllipticalDrop(fname='ed')
```

Note the addition of `fname='ed'`.  The filename and output directory to pick is determined by the filename and on IPython this is not meaningful, so if you want the output generated in the right directory, you must explicitly pass fname.  When run externally from Python, this is automatically determined for you.

### Exploring a bit more

- Learn more about the `WCSPHScheme`
- Learn about the `configure_solver` method and its options.

In [ ]:
from pysph.sph.scheme import WCSPHScheme
WCSPHScheme.configure_solver()

### Homework1

- Go over this tutorial: http://pysph.readthedocs.io/en/latest/tutorial/circular_patch_simple.html
- Familiarize yourself with it

## Advanced exercise

- Take the above example `ed.py` as a starting point and copy it to `dam_break.py`.
- Modify the `dam_break.py` to solve the 2D dam break problem, as below
- Fluid height = 2
- Fluid width = 1.0
- Container height = 4
- Container width = 4
- Assume g=9.81 m/s

- Use 2-3 layers of particles for the boundary


```
    |                         |
    |                         |           
    |   1                     |
    |*******                  |   4     
    |*******                  |
    |*******  2               |
    |*******                  |
    ---------------------------
    
                 4
```
    


- Create particles, add `solid` , `co`, `gy`, `hg_correction`, `update_h`, `nu` in `WCSPHScheme()`
- change `dt` accordingly : 0.125 * 10 * self.h / co
- Add `n_damp`, `adaptive_timestep` and `fixed_h` arguments in `configure_solver`
- Set `rho` for `solid` particles more than `fluids`

In [ ]:
import numpy as np

from pysph.solver.application import Application
from pysph.base.utils import get_particle_array
from pysph.sph.scheme import WCSPHScheme  
from matplotlib import pyplot as plt



class DamBreak2d(Application):
    def create_particles(self):
        
        ####### Geometry ######
        x = np.linspace(0,1,40)
        y = np.linspace(0,2,40)
        xm_fluid, ym_fluid = np.meshgrid(x, y)
        x_s_h = np.linspace(0.025/4,4,80)
        y_s_h = np.linspace(-0.025, 0, 3)
        x_s_v = np.linspace(-0.025, 0, 3)
        y_s_v = np.linspace(0,4,80)
        x_s_v_e = np.linspace(4, 4.025, 3)
        y_s_v_e = np.linspace(0,4,80)
        v_b_x, v_b_y = np.meshgrid(x_s_v, y_s_v)
        v_h_x, v_h_y = np.meshgrid(x_s_h, y_s_h)
        b3x, b3y = np.meshgrid(x_s_v_e, y_s_v_e)

        b1x, b1y = v_b_x, v_b_y
        b2x, b2y = v_h_x, v_h_y
        dx = 0.025
        h = 1.3*dx
        pa_solid_1 = get_particle_array(name='solid', x=b1x, y=b1y)
        pa_solid_2 = get_particle_array(name='solid', x=b2x, y=b2y)
        pa_solid_3 = get_particle_array(name='solid', x=b3x, y=b3y)
        bx, by = list(pa_solid_1.x.ravel())+list(pa_solid_2.x.ravel())+list(pa_solid_3.x.ravel()),\
        list(pa_solid_1.y.ravel())+list(pa_solid_2.y.ravel())+list(pa_solid_3.y.ravel())
        
        ###### Defining the arrays which are going to be used #######
        
        pa_solid = get_particle_array(name='solid', x=bx, y=by, rho=1100.0, h=h, m = 1100*dx*dx, p0=1000)
        pa_fluid = get_particle_array(name='fluid', x=xm_fluid+(0.25*dx), y=ym_fluid+(0.25*dx), \
                                      rho=1000.0, h=h, m = 1000*dx*dx, p0=1000)

        self.scheme.setup_properties([pa_fluid, pa_solid])
        return [pa_fluid, pa_solid]
    
        
    
    def create_scheme(self):
        s = WCSPHScheme(
            ['fluid'], ['solid'], dim=2, rho0=1000.0, c0=10*(40**0.5),
            h0=1.3*0.025, hdx=1.3, gamma=7.0, alpha=0.1, beta=0.0, gy=-9.81, hg_correction=True, update_h=True, nu=0.0, 
        )
        dt = 0.000644841269841269
        tf = 2.5
        s.configure_solver(
            dt=dt, tf=tf, n_damp=100, adaptive_timestep=True, fixed_h=False
        )
        return s 
        

    
app = DamBreak2d(fname='mysol_dam_break')
app.run()

In [ ]:
%matplotlib widget

from pysph.tools import ipy_viewer as ipy
new_sim = ipy.Viewer2D('mysol_dam_break_output/')
new_sim.interactive_plot()